In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# 1. IMPORT LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC

from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report

import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_csv("/kaggle/input/wine-quality/winequalityN.csv")

print(df.shape)
df.head()


(6497, 13)


,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
# Check missing values
df.isna().sum()

# Fill missing numeric values with median
for col in df.columns:
    if df[col].dtype != "object":
        df[col] = df[col].fillna(df[col].median())


In [4]:
X = df.drop(columns=["quality"])
y_reg = df["quality"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y_reg, test_size=0.2, random_state=42
)


In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


ValueError: could not convert string to float: 'white'

In [6]:
pca = PCA(n_components=8)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print("Explained variance ratio:", pca.explained_variance_ratio_.sum())


NameError: name 'X_train_scaled' is not defined

In [7]:
lr = LinearRegression()
lr.fit(X_train_pca, y_train)

pred_lr = lr.predict(X_test_pca)

print("Linear Regression RMSE:", np.sqrt(mean_squared_error(y_test, pred_lr)))
print("R²:", r2_score(y_test, pred_lr))


NameError: name 'X_train_pca' is not defined

In [8]:
rf = RandomForestRegressor(random_state=42)

param_grid_rf = {
    "n_estimators": [100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5]
}

grid_rf = GridSearchCV(rf, param_grid_rf, cv=3, n_jobs=-1, verbose=1)
grid_rf.fit(X_train_scaled, y_train)

print("Best RF Params:", grid_rf.best_params_)

pred_rf = grid_rf.best_estimator_.predict(X_test_scaled)

print("RF Regression RMSE:", np.sqrt(mean_squared_error(y_test, pred_rf)))
print("R²:", r2_score(y_test, pred_rf))


NameError: name 'X_train_scaled' is not defined

In [9]:
svr = SVR()

param_grid_svr = {
    "C": [0.1, 1, 10],
    "gamma": ["scale", "auto"],
    "kernel": ["rbf"]
}

grid_svr = GridSearchCV(svr, param_grid_svr, cv=3, n_jobs=-1, verbose=1)
grid_svr.fit(X_train_scaled, y_train)

print("Best SVR Params:", grid_svr.best_params_)

pred_svr = grid_svr.best_estimator_.predict(X_test_scaled)

print("SVR RMSE:", np.sqrt(mean_squared_error(y_test, pred_svr)))
print("R²:", r2_score(y_test, pred_svr)))


SyntaxError: unmatched ')' (1640536103.py, line 17)

In [ ]:
df["quality_class"] = df["quality"].apply(
    lambda x: 0 if x <= 5 else (1 if x == 6 else 2)
)

y_clf = df["quality_class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y_clf, test_size=0.2, random_state=42, stratify=y_clf
)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
rf_clf = RandomForestClassifier(random_state=42)

param_grid_clf = {
    "n_estimators": [100, 200],
    "max_depth": [None, 10, 20]
}

grid_rf_clf = GridSearchCV(rf_clf, param_grid_clf, cv=3, n_jobs=-1, verbose=1)
grid_rf_clf.fit(X_train_scaled, y_train)

print("Best RF Classifier Params:", grid_rf_clf.best_params_)

pred_rf_clf = grid_rf_clf.best_estimator_.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, pred_rf_clf))
print(classification_report(y_test, pred_rf_clf))


In [ ]:
svc = SVC()

param_grid_svc = {
    "C": [1, 10],
    "gamma": ["scale", "auto"],
    "kernel": ["rbf"]
}

grid_svc = GridSearchCV(svc, param_grid_svc, cv=3, n_jobs=-1, verbose=1)
grid_svc.fit(X_train_scaled, y_train)

print("Best SVC Params:", grid_svc.best_params_)

pred_svc = grid_svc.best_estimator_.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, pred_svc))
print(classification_report(y_test, pred_svc))


In [10]:
importances = grid_rf.best_estimator_.feature_importances_
names = X.columns

idx = np.argsort(importances)[::-1][:10]

plt.figure(figsize=(8,6))
sns.barplot(x=importances[idx], y=names[idx])
plt.title("Top 10 Important Features")
plt.show()


AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'